# Catchement area of farms

In [1]:
import os


def get_env_variable(var_name: str) -> str:
    try:
        value = os.environ[var_name]
        return value
    except KeyError:
        raise EnvironmentError(f"Missing required environment variable: {var_name}")

## define api key

In [2]:
api_key = get_env_variable("ORS_API_KEY")

## load building data

In [3]:
import geopandas as gpd

gdf = gpd.read_file('../data/processed_data/top_building_per_municipality.geojson')
print(gdf.shape)
gdf.head()

(27, 9)


,egid,buildingStatus,buildingCategory,buildingClass,municipalityNumber,municipalityName,nearby_farms,total_products,geometry
0,210197731,1004,1060,1252,159,Uetikon am See,"[ { ""farm_id"": 4.0, ""distance"": 5996.964112004...",802,POINT (8.68949 47.27389)
1,210202495,1004,1060,1252,155,MÃ¤nnedorf,"[ { ""farm_id"": 4.0, ""distance"": 6068.202328155...",788,POINT (8.69679 47.26797)
2,210213365,1004,1060,1252,157,Oetwil am See,"[ { ""farm_id"": 4.0, ""distance"": 8368.715941914...",782,POINT (8.72525 47.27579)
3,210101181,1004,1060,1252,121,Wetzikon (ZH),"[ { ""farm_id"": 11.0, ""distance"": 5166.19822639...",767,POINT (8.79499 47.31702)
4,210182556,1004,1060,1252,192,Egg,"[ { ""farm_id"": 4.0, ""distance"": 7268.427190714...",761,POINT (8.69620 47.28587)


## Create function to create isochrones

In [4]:
from openrouteservice import client
import json

# Client settings
ors = client.Client(key=api_key)


def create_isochrone(longitude: float, latitude: float, traveltime: int = 5, transport_mode = "cycling-regular") -> str:
    params_iso = {
        "profile": transport_mode,
        "range": [traveltime * 60],
        "attributes": ["total_pop"],
        "locations": [[longitude, latitude]],
    }

    try:
        result = ors.isochrones(**params_iso)
        return json.dumps(result)
    except Exception as e:
        print("Error creating isochrone:", e)
        return json.dumps({"error": str(e)})

## Create isochrones for buildings

### available profiles
* driving-car
* driving-hgv
* cycling-regular
* cycling-road
* cycling-mountain
* cycling-electric
* foot-walking
* foot-hiking
* wheelchair

In [5]:
import time

driving_times = [5, 10, 15]
transport_modes = ["cycling-regular", "foot-walking", "driving-car"]

# create isochrones for all buildings
for transport in transport_modes:
    for driving_time in [5, 10, 15]:
        for idx, row in gdf.iterrows():
            print(f'Processing building with index "{idx}, driving time "{driving_time} minutes" and transport mode "{transport}')
            try:
                result = create_isochrone(row["geometry"].x, row["geometry"].y, driving_time, transport)
                time.sleep(3) # wait for n seconds
                gdf.loc[idx, f"isochrone_within_{driving_time}_mode_{transport}"] = result
            except Exception as e:
                print(f"Failed to process building {idx}: {e}")
        print(f"Finished processing isochrones for {driving_time} minutes with transport mode {transport}")
    print(f"Finished processing isochrones for transport mode {transport}")

gdf.to_file('../data/processed_data/top_building_per_municipality_with_isochrones.geojson', driver='GeoJSON')

Processing building with index "0, driving time "5 minutes" and transport mode "cycling-regular
Processing building with index "1, driving time "5 minutes" and transport mode "cycling-regular
Processing building with index "2, driving time "5 minutes" and transport mode "cycling-regular
Processing building with index "3, driving time "5 minutes" and transport mode "cycling-regular
Processing building with index "4, driving time "5 minutes" and transport mode "cycling-regular
Processing building with index "5, driving time "5 minutes" and transport mode "cycling-regular
Processing building with index "6, driving time "5 minutes" and transport mode "cycling-regular
Processing building with index "7, driving time "5 minutes" and transport mode "cycling-regular
Processing building with index "8, driving time "5 minutes" and transport mode "cycling-regular
Processing building with index "9, driving time "5 minutes" and transport mode "cycling-regular
Processing building with index "10, driv

## create a summary table

In [22]:
def extract_population(row, column_name):
    try:
        return json.loads(row[column_name])['features'][0]['properties']['total_pop']
    except (KeyError, TypeError, json.JSONDecodeError):
        return None

# List of isochrone columns in your DataFrame
isochrone_columns = [col for col in gdf.columns if col.startswith('isochrone_within')]

# Extract population data for each isochrone column
for col in isochrone_columns:
    gdf[f'{col}_pop'] = gdf.apply(lambda row: extract_population(row, col), axis=1)

# Create a summary table with total populations by municipality and isochrone
municipality_iso_pop = gdf.groupby('municipalityName')[[f'{col}_pop' for col in isochrone_columns]].sum().reset_index()

# Rename columns for clarity in the summary table
column_renames = {f'{col}_pop': col for col in isochrone_columns}
municipality_iso_pop = municipality_iso_pop.rename(columns=column_renames)

# remove columns with suffix _pop
municipality_iso_pop = municipality_iso_pop[[col for col in municipality_iso_pop.columns if not col.endswith('_pop')]]

print(municipality_iso_pop.head())

  municipalityName  isochrone_within_5_mode_cycling-regular  \
0    BachenbÃ¼lach                                   4014.0   
1          BÃ¼lach                                  15019.0   
2        DÃ¤gerlen                                    582.0   
3              Egg                                   1843.0   
4      Gossau (ZH)                                   4142.0   

   isochrone_within_10_mode_cycling-regular  \
0                                   26145.0   
1                                   27776.0   
2                                    4715.0   
3                                   15474.0   
4                                   23380.0   

   isochrone_within_15_mode_cycling-regular  \
0                                   35374.0   
1                                   37449.0   
2                                   11367.0   
3                                   37841.0   
4                                   38925.0   

   isochrone_within_5_mode_foot-walking  \
0           

In [23]:
municipality_iso_pop.to_csv('../data/processed_data/municipality_isochrone_population.csv', index=False)

## filter the buildings with the highest population

In [15]:
def sort_gdf_by_population_and_rename(gdf, column_name, amount=5):
    # Convert the column's stringified JSON to actual JSON and then extract the population data
    gdf["total_pop"] = gdf.apply(
        lambda row: (
            json.loads(row[column_name])["features"][0]["properties"]["total_pop"]
            if row[column_name] is not None
            else None
        ),
        axis=1,
    )

    # Remove rows where population data is not available
    gdf = gdf.dropna(subset=["total_pop"])

    # Sort the DataFrame based on the total population in descending order
    sorted_gdf = gdf.sort_values(by="total_pop", ascending=False)

    # Drop all columns starting with "isochrone" except the one used for sorting
    columns_to_keep = [
        col
        for col in sorted_gdf.columns
        if not col.startswith("isochrone") or col == column_name
    ]
    sorted_gdf = sorted_gdf[columns_to_keep]

    # Rename the remaining isochrone column to "isochrone_data"
    sorted_gdf = sorted_gdf.rename(columns={column_name: "isochrone_data"})

    return sorted_gdf.head(amount)

In [16]:
sorted_gdf = sort_gdf_by_population_and_rename(gdf, 'isochrone_within_15_mode_cycling-regular')

## display some statistics

## display with folium

In [17]:
import folium
import json

column_name = "isochrone_data"

map_center = [47.3769, 8.5417]  # Default to Zurich, Switzerland

m = folium.Map(location=map_center, zoom_start=12, tiles="CartoDB positron")

# Step 2: Add isochrones to the map
for idx, row in sorted_gdf.iterrows():
    if column_name in row and row[column_name]:
        try:
            # Parse the JSON string back to a dictionary
            isochrone_data = json.loads(row[column_name])

            # Use Folium's GeoJson to add it to the map
            folium.GeoJson(
                isochrone_data,
                name=f"Isochrone {idx}",
                style_function=lambda feature: {
                    "fillColor": "#blue",
                    "color": "blue",
                    "weight": 2,
                    "dashArray": "5, 5",
                    "fillOpacity": 0.5,
                },
            ).add_to(m)
        except json.JSONDecodeError:
            print(f"Failed to decode JSON for row {idx}")

# Add layer control to toggle layers
folium.LayerControl().add_to(m)

# Display the map
m